# 🍎 AIProjectClient를 사용한 채팅 완료 🍏

이 노트북에서는 **Azure AI Foundry SDK**를 사용하여 **채팅 완성(Chat Completion)**을 수행하는 방법을 보여드리겠습니다. **`azure-ai-projects`** 및 **`azure-ai-inference`** 패키지를 결합하여 다음과 같이 합니다:

1. `AIProjectClient`를 **초기화**합니다.
2. 직접 LLM 호출을 수행할 채팅 완료 클라이언트를 **가져옵니다**.
3. **프롬프트 템플릿**을 **사용**하여 시스템 컨텍스트를 추가합니다.
4. 건강 및 피트니스 테마로 사용자 프롬프트를 **보냅니다**.

## 🏋️ 건강-피트니스 면책 조항
> **이 예시는 데모용이며 실제 의학적 조언을 제공하지 않습니다**. 건강 또는 의료 관련 질문은 항상 전문가와 상담하세요.

### 전제 조건
이 노트북을 시작하기 전에 [README.md](../../README.md#-prerequisites)에 나열된 모든 사전 요구 사항을 완료했는지 확인하세요.

그럼 시작해 보겠습니다! 🎉

<img src="./seq-diagrams/1-chat.png" width="30%"/>


## 1. 초기 설정
환경 변수를 로드하고, `AIProjectClient`를 만들고, `ChatCompletionsClient`를 가져옵니다. 또한 **프롬프트 템플릿**을 정의하여 시스템 메시지를 구성하는 방법을 보여줍니다.


In [ ]:
import os
from dotenv import load_dotenv
from pathlib import Path
from azure.identity import DefaultAzureCredential

from azure.ai.projects import AIProjectClient
from azure.ai.inference.models import UserMessage, SystemMessage  # for chat messages

# Load environment variables
notebook_path = Path().absolute()
parent_dir = notebook_path.parent
load_dotenv(parent_dir / '.env')

# Retrieve from environment
connection_string = os.environ.get("PROJECT_CONNECTION_STRING")
model_deployment = os.environ.get("MODEL_DEPLOYMENT_NAME")

try:
    # Create the project client
    project_client = AIProjectClient.from_connection_string(
        credential=DefaultAzureCredential(),
        conn_str=connection_string,
    )
    print("✅ Successfully created AIProjectClient")
except Exception as e:
    print("❌ Error initializing client:", e)


### 프롬프트 템플릿
면책 조항을 제공하는 친절한 피트니스 도우미로서 컨텍스트를 설정하는 빠른 **시스템** 메시지를 정의하겠습니다.

```txt
SYSTEM PROMPT (template):
You are FitChat GPT, a helpful fitness assistant.
Always remind users: I'm not a medical professional.
Be friendly, provide general advice.
...
```

그런 다음 사용자 콘텐츠를 **사용자** 메시지로 전달합니다.


In [ ]:
# We'll define a function that runs chat completions with a system prompt & user prompt
def chat_with_fitness_assistant(user_input: str):
    """Use chat completions to get a response from our LLM, with system instructions."""
    # Our system message template
    system_text = (
        "You are FitChat GPT, a friendly fitness assistant.\n"
        "Always remind users: I'm not a medical professional.\n"
        "Answer with empathy and disclaimers."
    )

    # We'll open the chat completions client
    with project_client.inference.get_chat_completions_client() as chat_client:
        # Construct messages: system + user
        system_message = SystemMessage(content=system_text)
        user_message = UserMessage(content=user_input)

        # Send the request
        response = chat_client.complete(
            model=model_deployment,
            messages=[system_message, user_message]
        )

        return response.choices[0].message.content  # simplest approach: get top choice's content

print("Defined a helper function to do chat completions.")

## 2. 채팅 완성 기능 사용해보기 🎉
건강이나 피트니스에 관한 사용자 질문으로 함수를 호출하고 결과를 확인해 보겠습니다. 질문을 자유롭게 수정하거나 여러 번 실행해 보세요!


In [ ]:
user_question = "How can I start a beginner workout routine at home?"
reply = chat_with_fitness_assistant(user_question)
print("🗣️ User:", user_question)
print("🤖 Assistant:", reply)

## 3. 다른 예제: 입력란이 있는 프롬프트 템플릿 📝
조금 더 나아가 시스템 메시지에 자리 표시자를 추가할 수 있습니다. 예를 들어 **userName** 또는 **goal**이 있다고 가정해 보겠습니다. 최소한의 예시를 보여드리겠습니다.


In [ ]:
def chat_with_template(user_input: str, user_name: str, goal: str):
    # Construct a system template with placeholders
    system_template = (
        "You are FitChat GPT, an AI personal trainer for {name}.\n"
        "Your user wants to achieve: {goal}.\n"
        "Remind them you're not a medical professional. Offer friendly advice."
    )

    # Fill in placeholders
    system_prompt = system_template.format(name=user_name, goal=goal)

    with project_client.inference.get_chat_completions_client() as chat_client:
        system_msg = SystemMessage(content=system_prompt)
        user_msg = UserMessage(content=user_input)

        response = chat_client.complete(
            model=model_deployment,
            messages=[system_msg, user_msg]
        )

    return response.choices[0].message.content

# Let's try it out
templated_user_input = "What kind of home exercise do you recommend for a busy schedule?"
assistant_reply = chat_with_template(
    templated_user_input,
    user_name="Jordan",
    goal="increase muscle tone and endurance"
)
print("🗣️ User:", templated_user_input)
print("🤖 Assistant:", assistant_reply)

## 🎉 축하합니다!
Azure AI Foundry의 `AIProjectClient` 및 `azure-ai-inference`를 사용하여 **채팅 완료**를 성공적으로 수행했습니다. 또한 **프롬프트 템플릿**을 통합하여 시스템 지침을 맞춤화하는 방법도 살펴보았습니다.

#### 워크샵의 다음 부분을 보려면 [2-embeddings.ipynb](2-embeddings.ipynb)로 이동하세요!  🎯